In [17]:
import pandas as pd
import pystan

In [16]:
firewood_code = """

data {
    # samples per row by species
    int totals1[3];
    int totals2[3];
    int totals3[3];
    int totals4[3];
}

parameters {
    # probabilities per row of each species
    vector[3] p_species1;
    vector[3] p_species2;
    vector[3] p_species3;
    vector[3] p_species4; 
}

model {
    # row 1 samples by species
    p_species1[1] ~ beta(5, 5); # row 1 alder prior
    p_species1[2] ~ beta(2, 8); # row 1 birch prior
    p_species1[3] ~ beta(3, 7); # row 1 pine prior
    totals1 ~ multinomial(softmax(p_species1));
    
    # row 2 samples by species
    p_species2[1] ~ beta(7, 3); # row 2 alder prior
    p_species2[2] ~ beta(2, 8); # row 2 birch prior
    p_species2[3] ~ beta(1, 9); # row 2 pine prior
    totals2 ~ multinomial(softmax(p_species2));
    
    # row 3 samples by species
    p_species3[1] ~ beta(15, 5); # row 3 alder prior
    p_species3[2] ~ beta(4, 16); # row 3 birch prior
    p_species3[3] ~ beta(1, 19); # row 3 pine prior
    totals3 ~ multinomial(softmax(p_species3));
    
    # row 4 samples by species
    p_species4[1] ~ beta(15, 5); # row 4 alder prior
    p_species4[2] ~ beta(4, 16); # row 4 birch prior
    p_species4[3] ~ beta(1, 19); # row 4 pine prior
    totals4 ~ multinomial(softmax(p_species4));
}
"""
firewood_model = pystan.StanModel(model_code=firewood_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_a1d3d35bdebce0a10713daab5ca8333f NOW.
